In [1]:
import torch
import pandas as pd
import numpy as np
import os
import torch.nn.functional as F
import copy
import matplotlib as mpl
import matplotlib.pyplot as plt
from ipywidgets import interact
from os import listdir
from os.path import isfile, join


path = os.path.join(os.getcwd())
dataset_path = os.path.join(path, "datasets", "full_corrected_dataset", "test")
experiments_path = os.path.join(path, "experiments")

filenames = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]
experiments = [f for f in listdir(experiments_path) if not isfile(join(experiments_path, f))]
models = ["best", "final"]

@interact(test_filename_id=(0,len(filenames)-1,1), experiment=experiments, bestOrFinal=models)
def test_model(test_filename_id=1,experiment=1,bestOrFinal=1):
    filename = filenames[test_filename_id]
    experiment_name = experiment
    model = bestOrFinal
    model_path = os.path.join(experiments_path, experiment_name, "model_weights", "model_" + model + ".pth")

    mapping_path = os.path.join(path, "datasets", "full_corrected_dataset_to_RAW_mapping.csv")
    file_path = os.path.join(dataset_path, filename)

    mapping = pd.read_csv(mapping_path, sep=";")
    data_csv = pd.read_csv(file_path)
    
    original_filename = mapping.loc[mapping['new_id'] == filename].values[:, 2][0]
    tensors = []
    padding_minimum = torch.zeros(180)
    
    if "4cls" in experiment_name:
        names = ["T1", "T2", "T3", "T4"]
    else:
        names = ["T1", "T2", "T3", "T4", "A+E"]

    data = data_csv.iloc[:, 1:].values[:, 0]
    data = data - np.min(data)
    data = data / np.max(data)
    input_tensor = torch.tensor(data, dtype=torch.double)

    tensors.append(input_tensor)
    tensors.append(padding_minimum)
    tensors = torch.nn.utils.rnn.pad_sequence(tensors, batch_first=True)
    tensors = tensors[:-1]

    model = torch.load(model_path,map_location=torch.device('cpu'))
    model.eval()
    outputs = model(tensors)
    softmax_outputs = F.softmax(outputs, dim=1).tolist()[0]
    
    print("Filename: {}\n".format(str(filename)),
         "Original file location: {}\n".format(str(original_filename)),
         "Model: {}\n{}\n".format(str(experiment_name), model),
         "Original class: {}\n".format(filename.split("_")[0]),
         "Predicted class: {}\n".format(names[np.argmax(softmax_outputs, axis=0)]),
         "Predicted probability: {}\n".format(np.max(softmax_outputs)))
    
    plt.figure(figsize=(12,12))
    plt.subplot(211)
    plt.plot(data_csv.iloc[:, 0:].values[:, 0],data_csv.iloc[:, 1:].values[:, 0])

    plt.subplot(212)
    plt.bar(names, softmax_outputs)
    plt.show()


interactive(children=(IntSlider(value=1, description='test_filename_id', max=1850), Dropdown(description='expe…